In [3]:
import numpy as np
import tensorflow as tf
import keras
from keras import models
from keras import layers
from keras.layers import Input, Lambda, Conv3D, Bidirectional, LSTM, Dense, TimeDistributed, Attention, Reshape,Conv1D

SINGLE_ATTENTION_VECTOR = True
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = inputs
    #a = Permute((2, 1))(inputs)
    #a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(input_dim, activation='softmax')(a)
    
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: keras.ops.mean(x, axis=1), name='dim_reduction')(a)
        a = layers.RepeatVector(input_dim)(a)
    a_probs = layers.Permute((1, 2), name='attention_vec')(a)

    #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = layers.Multiply()([inputs, a_probs])
    return output_attention_mul

# 假设有1000个样本，每个样本有6个特征，时间步长为10
timesteps = 10
features = 6
num_classes = 3

# 生成随机数据
data = np.random.rand(1000, timesteps, features)
labels = np.random.randint(0, num_classes, (1000,))

# 一热编码标签
one_hot_labels = keras.utils.to_categorical(labels, num_classes)

# 定义输入
input_layer = Input(shape=(timesteps, features))

# 3D卷积层
conv1d_layer = Conv1D(filters=64, kernel_size=(3), activation='relu')(input_layer)

v = layers.MaxPool1D(pool_size=2)(conv1d_layer)
# BiLSTM层
bilstm_output = Bidirectional(LSTM(units=64, return_sequences=True, return_state=False))(v)

query_dim =features
# 假设你的模型不需要键（key），只需要查询（query）和值（value）
# 你可以简单地将 LSTM 层的输出分成两半
queries = bilstm_output[:, :, :query_dim]
values = bilstm_output[:, :, query_dim:]
# 注意力层
attention_layer = Attention()([queries, values])

# 时间分布的全连接层
time_distributed_layer = TimeDistributed(Dense(units=64, activation='relu'))(x)

# 全连接层，用于分类
output_layer = Dense(units=num_classes, activation='softmax')(time_distributed_layer)

# 创建模型
model = models.Model(inputs=input_layer, outputs=output_layer)

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 打印模型概要
model.summary()

# 查看模型结构
model.get_config()

# 训练模型
model.fit(data, one_hot_labels, epochs=10, batch_size=32)

NameError: name 'x' is not defined